# ATLAS/ICESat-2 L3A Land and Vegetation Height, Version 5 (ATL08)

Author: Sumant Jha
Date: 04/07/2023

This data set (ATL08) contains along-track heights above the WGS84 ellipsoid (ITRF2014 reference frame) for the ground and canopy surfaces. The canopy and ground surfaces are processed in fixed 100 m data segments, which typically contain more than 100 signal photons. The data were acquired by the Advanced Topographic Laser Altimeter System (ATLAS) instrument on board the Ice, Cloud and land Elevation Satellite-2 (ICESat-2) observatory.

```
Parameter(s): CANOPY HEIGHT TERRAIN ELEVATION
Platform(s):ICESat-2
Sensor(s): ATLAS
Data Format(s): HDF5
Temporal Coverage: 14 October 2018 to present
Temporal Resolution: 91 day
Spatial Resolution: Varies
Spatial Reference System(s): WGS 84 EPSG:4326
Spatial Coverage: N: 90 S: -90 E: 180 W: -180
            
(source: https://nsidc.org/data/atl08/versions/5)
```


There are two examples shown below which will demostrate how to access ATL08 data.The first example shows how we can download it to local from NASA CMR and the second example shows how we can use the AWS href link to work on the data without downloading to local. 

## Example 1: Download data to local

### Import relevant packages 

The following example uses: maap-py,h5py, and h5glance.
If you do not have these packages, use:

```
!pip install maap-py h5py h5glance requests rioxarray rasterio
```

In [1]:
import os
import h5py
from maap.maap import MAAP
from h5glance import H5Glance
import requests
import rioxarray
import rasterio as rio
from rasterio.session import AWSSession

Now that we have imported relevant packages, lets put them to use. We are going to use NASA host which is NASA's Common Metadata Repository (NASA-CMR) to search for and download ICESat data. 
ICESat's ATL08 data's concept id can be found on https://search.earthdata.nasa.gov/search and looking for 'ATL08' in the search bar. When you check the metadata associated with your search result, you can get the concept_id associated with below tutorial. In this case, the concept_id is `C2153574670-NSIDC_CPRD`.  

For this example, we are going to use granule id of `ATL08_20211114213015_08161305_005_01`. This will be in HDF5 format. 

With all this information in hand, we are ready to make a query to cmr.earthdata.nasa.gov using maap-py. 


In [2]:
maap = MAAP(maap_host='api.ops.maap-project.org')

nasa_host = "cmr.earthdata.nasa.gov"
results = maap.searchGranule(cmr_host=nasa_host,
                             concept_id="C2153574670-NSIDC_CPRD", 
                             readable_granule_name="ATL08_20211114213015_08161305_005_01.h5", 
                             limit=100)

Let's see how this turned out. Did we get a result?

In [3]:
results[0]

{'concept-id': 'G2208041838-NSIDC_CPRD',
 'collection-concept-id': 'C2153574670-NSIDC_CPRD',
 'revision-id': '1',
 'format': 'application/echo10+xml',
 'Granule': {'GranuleUR': 'ATL08_20211114213015_08161305_005_01.h5',
  'InsertTime': '2022-01-26T23:08:01.838Z',
  'LastUpdate': '2022-01-26T23:08:01.838Z',
  'Collection': {'DataSetId': 'ATLAS/ICESat-2 L3A Land and Vegetation Height V005'},
  'DataGranule': {'SizeMBDataGranule': '15.952223777770996',
   'ProducerGranuleId': 'ATL08_20211114213015_08161305_005_01.h5',
   'DayNightFlag': 'UNSPECIFIED',
   'ProductionDateTime': '2021-12-21T01:54:36.000Z'},
  'Temporal': {'RangeDateTime': {'BeginningDateTime': '2021-11-14T21:30:15.724Z',
    'EndingDateTime': '2021-11-14T21:35:25.694Z'}},
  'Spatial': {'HorizontalSpatialDomain': {'Orbit': {'AscendingCrossing': '-169.8753286285306',
     'StartLat': '80',
     'StartDirection': 'D',
     'EndLat': '59.5',
     'EndDirection': 'D'}}},
  'OrbitCalculatedSpatialDomains': {'OrbitCalculatedSpatial

Okay, we did get our result and we learnt a lot about it from available metadata. Let's download the HDF file and check the available keys and structure of it, using H5py and H5glance. 

In [4]:
results[0]['Granule']['OnlineAccessURLs']['OnlineAccessURL'][1]['URL']

's3://nsidc-cumulus-prod-protected/ATLAS/ATL08/005/2021/11/14/ATL08_20211114213015_08161305_005_01.h5'

In [5]:
data_file = results[0]

Establish a temporary directory to store the data file and display the path and filename. 

In [6]:
dataDir = '/projects/tmp'
if not os.path.exists(dataDir): os.mkdir(dataDir)
data = data_file.getData(dataDir)
data

'/projects/tmp/ATL08_20211114213015_08161305_005_01.h5'

Open the H5 file and list the keys

In [7]:
atl08_file = h5py.File(data,'r')
list(atl08_file.keys())

['METADATA',
 'ancillary_data',
 'ds_geosegments',
 'ds_metrics',
 'ds_surf_type',
 'gt1l',
 'gt1r',
 'gt2l',
 'gt2r',
 'gt3l',
 'gt3r',
 'orbit_info',
 'quality_assessment']

Use H5glance module to interactively check all available variables and field that can be used for further analysis and visualizations.

In [8]:
H5Glance(atl08_file)

/projects/tmp/ATL08_20211114213015_08161305_005_01.h5/ (47 attributes)
├METADATA	(9 children) (3 attributes)
├ancillary_data	(26 children) (2 attributes)
├ds_geosegments	[int8: 5] (12 attributes)
├ds_metrics	[int8: 18] (12 attributes)
├ds_surf_type	[int32: 5] (12 attributes)
├gt1l	(2 children) (7 attributes)
├gt1r	(2 children) (7 attributes)
├gt2l	(2 children) (7 attributes)
├gt2r	(2 children) (7 attributes)
├gt3l	(2 children) (7 attributes)
├gt3r	(2 children) (7 attributes)
├orbit_info	(7 children) (2 attributes)
└quality_assessment	(2 children) (1 attributes)

## Example 2: Accessing file without downloading, but by setting up an AWS instance. 

### In this example, we do not actually download the h5 file. In this example we:
1. Access the data using S3 endpoints, 
2. Pass temporary credentials to boto3, 
3. Use rasterios to create an AWS session, 
4. Use the S3 URL to access data, and 
5. Finally use rioxarray to open the dataset.

Generating S3 Credential endpoint for ICESat datasets.

In [9]:
s3_cred_endpoint = {
    'nsidc': 'https://data.nsidc.earthdatacloud.nasa.gov/s3credentials'
}

Function to get temporary credentials using 'request' module. 

In [10]:
def get_temp_creds(provider):
    return requests.get(s3_cred_endpoint[provider]).json()

In [ ]:
temp_creds_req = get_temp_creds('nsidc')

In [76]:
import boto3
session = boto3.Session(aws_access_key_id=temp_creds_req['accessKeyId'], 
                        aws_secret_access_key=temp_creds_req['secretAccessKey'],
                        aws_session_token=temp_creds_req['sessionToken'],
                        region_name='us-west-2')

In [77]:
rio_env = rio.Env(AWSSession(session),
                  GDAL_DISABLE_READDIR_ON_OPEN='TRUE',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
rio_env.__enter__()

In [78]:
s3_url = 's3://nsidc-cumulus-prod-protected/ATLAS/ATL08/005/2021/11/14/ATL08_20211114213015_08161305_005_01.h5'

Alternatively, we can use part of the result from Example 1, to get the S3 URL. 

In [34]:
#s3_url = results[0]['Granule']['OnlineAccessURLs']['OnlineAccessURL'][1]['URL']

In [ ]:
da = rioxarray.open_rasterio(s3_url)
da